In [1]:
!gdown --id 1y53EsKCX2B4XNFV74sbyibUKgfAm34Td
# https://huggingface.co/docs/transformers/tasks/multiple_choice#inference

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1y53EsKCX2B4XNFV74sbyibUKgfAm34Td
To: /content/math_train.json
100% 493k/493k [00:00<00:00, 145MB/s]


In [2]:
!pip install -q datasets
!pip install -q -U accelerate
!pip install -q -U transformers
!pip install -q sentencepiece
!pip install peft
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import os
import json
import torch
from typing import Optional, Union
import numpy as np
from datasets import Dataset
from dataclasses import dataclass
from transformers import AutoTokenizer
from transformers import EarlyStoppingCallback
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

In [4]:
 from peft import LoraConfig, get_peft_model, TaskType

In [5]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
%env WANDB_PROJECT=zaic_2023_math_v6

env: WANDB_PROJECT=zaic_2023_math_v6


In [7]:
with open("math_train.json") as json_file:
    json_data = json.load(json_file)
    print(json_data['data'])

[{'id': '1', 'question': 'Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính', 'choices': ['A. 80,000 : 6', 'B. 80,000 x 6', 'C. 80,000 : (6 x 100)', 'D. (80,000 x 6) : 100'], 'explanation': 'Theo đề bài, số tiền lỗ bằng 6% của 80 000 đồng . Để tìm số tiền lỗ ta có thể lấy 80 000 chia cho 100 rồi nhân với 6 (tức là 80 000 : 100 × 6) hoặc lấy 80000 nhân với 6 rồi chia cho 100 (tức là 80 000 × 6 : 100).', 'answer': 'D. (80,000 x 6) : 100'}, {'id': '10', 'question': '8 dm2 24 cm2 = ……… dm2. Số thích hợp điền vào chỗ chấm là:', 'choices': ['A. 824', 'B. 82,4', 'C. 8,24', 'D. 0,824'], 'explanation': 'Ta có 24 cm2 = 0,24 dm2\n Vậy 8 dm2 24 cm2 = 8,24 dm2.', 'answer': 'C. 8,24'}, {'id': '100', 'question': '10% của 11,5m2 là:', 'choices': ['A. 10,15dm2', 'B. 1,5m2', 'C. 15,5m2', 'D. 1,15m2'], 'explanation': '10% của 11,5m2 là: 11,5 ${\\times}$ 10 : 100 = 1,15 (m2).', 'answer': 'D. 1,15m2'}, {'id': '1004', 'question': 'Một tam giác có đáy bằng 15 

In [8]:
dict_one_hot_answer = {
    0: "A",
    1: "B",
    2: "C",
    3: "D"
}

list_question = []
list_answer = []
list_A = []
list_B = []
list_C = []
list_D = []
list_explanation = []

for record in json_data['data']:
  question = record['question']
  choices = record['choices']
  try:
    explanation = record['explanation']
  except KeyError:
    explanation = "None"
  answer = record['answer']

  list_A.append(choices[0])
  list_B.append(choices[1])
  list_C.append(choices[2])
  try:
    list_D.append(choices[3])
  except IndexError:
    list_D.append("None")
  list_question.append(question)
  one_hot_answer = choices.index(answer)
  list_answer.append(dict_one_hot_answer[one_hot_answer])
  list_explanation.append(explanation)

In [9]:
data_df = pd.DataFrame(list(zip(list_question, list_explanation, list_A, list_B, list_C, list_D, list_answer)),
                       columns=['question', 'explanation', 'A', 'B', 'C', 'D', 'answer'])

In [10]:
data_df

,question,explanation,A,B,C,D,answer
0,"Một người bán hàng bỏ ra 80,000 đồng tiền vốn ...","Theo đề bài, số tiền lỗ bằng 6% của 80 000 đồn...","A. 80,000 : 6","B. 80,000 x 6","C. 80,000 : (6 x 100)","D. (80,000 x 6) : 100",D
1,8 dm2 24 cm2 = ……… dm2. Số thích hợp điền v...,"Ta có 24 cm2 = 0,24 dm2\n Vậy 8 dm2 24 cm2 = ...",A. 824,"B. 82,4","C. 8,24","D. 0,824",C
2,"10% của 11,5m2 là:","10% của 11,5m2 là: 11,5 ${\times}$ 10 : 100 = ...","A. 10,15dm2","B. 1,5m2","C. 15,5m2","D. 1,15m2",D
3,"Một tam giác có đáy bằng 15 cm và gấp 2,5 l...","Chiều cao của tam giác đó là: 15 : 2,5 = 6 (c...",A. 90 cm2,"B. 37,5 cm2",C. 45 cm2,"D. 18,75 cm2",C
4,Một tấm thiệp hình chữ nhật có chiều rộ...,Chiều dài hình chữ nhật là: 9 ${\times}$ 2 ...,A. 162 cm2,B. 81 cm2,C. 99 cm2,D. 63 cm2,A
...,...,...,...,...,...,...,...
1195,"Một ô tô đi được 60 km với vận tốc 60 km/giờ, ...","vì đoạn đường thứ nhất ô tô đi hết 1 giờ, đoạn...","A. 1,5 giờ",B. 2 giờ,C. 3 giờ,D. 4 giờ,C
1196,"Một ô tô đi từ A đến B với vận tốc 38,5k...","Thời gian ô tô đã đi là: 100,1 : 38,5 = 2,6 (...",A. 2 giờ 1 phút,B. 2 giờ 6 phút,C. 2 giờ 36 phút,D. 2 giờ 30 phút,C
1197,Một ô tô đi từ Hà Nội lúc 11 giờ trưa và đến V...,5 giờ 30 phút chiều = 17 giờ 30 phút\n Thời gi...,A. 4 giờ 30 phút,B. 6 giờ 30 phút,C. 6 giờ 15 phút,D. 6 giờ,D
1198,Một sân vận động hình chữ nhật có chiều ...,Chiều dài sân vận động là: 95 ${\times}$ 3 ...,A.380 m,B.285 m,C.670 m,D.760 m,D


# Config

In [11]:
VER=6
# TRAIN WITH SUBSET OF 60K
NUM_TRAIN_SAMPLES = 512
# PARAMETER EFFICIENT FINE TUNING
# PEFT REQUIRES 1XP100 GPU NOT 2XT4
USE_PEFT = False
# NUMBER OF LAYERS TO FREEZE
# DEBERTA LARGE HAS TOTAL OF 24 LAYERS
FREEZE_LAYERS = 21
# BOOLEAN TO FREEZE EMBEDDINGS
FREEZE_EMBEDDINGS = True
# LENGTH OF CONTEXT PLUS QUESTION ANSWER
MAX_INPUT = 256
# HUGGING FACE MODEL
MODEL = 'microsoft/deberta-v3-large'

# Dataloader

In [12]:
option_to_index = {option: idx for idx, option in enumerate('ABCD')}
index_to_option = {v: k for k,v in option_to_index.items()}

def preprocess(example):
    first_sentence = [ "[CLS] " + example['explanation'] ] * 4
    second_sentences = [" #### " + example['question'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCD']
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation=True,
                                  max_length=MAX_INPUT, add_special_tokens=False)
    tokenized_example['label'] = option_to_index[example['answer']]

    return tokenized_example

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [13]:
from sklearn.model_selection import train_test_split

df_train, df_valid = train_test_split(data_df, test_size=0.1, random_state=42)

print(df_train.shape, df_valid.shape)

(1080, 7) (120, 7)


In [14]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
dataset_valid = Dataset.from_pandas(df_valid)
dataset = Dataset.from_pandas(df_train)
dataset = dataset.remove_columns(["__index_level_0__"])
dataset

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Dataset({
    features: ['question', 'explanation', 'A', 'B', 'C', 'D', 'answer'],
    num_rows: 1080
})

In [15]:
tokenized_dataset_valid = dataset_valid.map(preprocess, remove_columns=['question', 'explanation', 'A', 'B', 'C', 'D', 'answer'])
tokenized_dataset = dataset.map(preprocess, remove_columns=['question', 'explanation', 'A', 'B', 'C', 'D', 'answer'])
tokenized_dataset

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/1080 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 1080
})

# Build model

In [16]:
model = AutoModelForMultipleChoice.from_pretrained(MODEL)

pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
if USE_PEFT:
    print('We are using PEFT.')
    from peft import LoraConfig, get_peft_model, TaskType
    peft_config = LoraConfig(
        r=8, lora_alpha=4, task_type=TaskType.SEQ_CLS, lora_dropout=0.1,
        bias="none", inference_mode=False,
        target_modules=["query_proj", "value_proj"],
        modules_to_save=['classifier','pooler'],
    )
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

In [18]:
if FREEZE_EMBEDDINGS:
    print('Freezing embeddings.')
    for param in model.deberta.embeddings.parameters():
        param.requires_grad = False
if FREEZE_LAYERS>0:
    print(f'Freezing {FREEZE_LAYERS} layers.')
    for layer in model.deberta.encoder.layer[:FREEZE_LAYERS]:
        for param in layer.parameters():
            param.requires_grad = False

Freezing embeddings.
Freezing 21 layers.


In [19]:
def map_at_3(predictions, labels):
    map_sum = 0
    pred = np.argsort(-1*np.array(predictions),axis=1)[:,:3]
    for x,y in zip(pred,labels):
        z = [1/i if y==j else 0 for i,j in zip([1,2,3],x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)

def compute_metrics(p):
    predictions = p.predictions.tolist()
    labels = p.label_ids.tolist()
    return {"map@3": map_at_3(predictions, labels)}

In [20]:
training_args = TrainingArguments(
    warmup_ratio=0.1,
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    num_train_epochs=100,
    report_to='wandb',
    output_dir = f'./checkpoints_{VER}',
    overwrite_output_dir=True,
    fp16=True,
    gradient_accumulation_steps=8,
    logging_steps=25,
    evaluation_strategy='steps',
    eval_steps=25,
    save_strategy="steps",
    save_steps=25,
    load_best_model_at_end=False,
    metric_for_best_model='map@3',
    lr_scheduler_type='cosine',
    weight_decay=0.01,
    save_total_limit=2,
)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset_valid,
    compute_metrics = compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

trainer.train()
trainer.save_model(f'model_v{VER}')
torch.save(model.state_dict(), f'/content/model_v{VER}/pytorch_model.bin')

wandb: Currently logged in as: octoopt. Use `wandb login --relogin` to force relogin


You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Map@3
25,1.385900,1.386336,0.452778
50,1.388000,1.386125,0.545833
75,1.389200,1.385848,0.630556
100,1.387100,1.385490,0.673611
125,1.387300,1.384538,0.686111
150,1.386100,1.379175,0.702778
175,1.314000,0.991690,0.729167
200,0.906200,0.878217,0.701389
225,0.863900,0.853782,0.712500
250,0.829000,0.854345,0.731944


# Predict

In [22]:
!gdown 1C_OJ6F-BEEvdZI1dcLTa9r5dltly9gpc

Downloading...
From: https://drive.google.com/uc?id=1C_OJ6F-BEEvdZI1dcLTa9r5dltly9gpc
To: /content/math_test.json
100% 53.9k/53.9k [00:00<00:00, 108MB/s]


In [23]:
del model, trainer
if USE_PEFT:
    model = AutoModelForMultipleChoice.from_pretrained(MODEL)
    model = get_peft_model(model, peft_config)
    checkpoint = torch.load(f'/content/model_v{VER}/pytorch_model.bin')
    model.load_state_dict(checkpoint)
else:
    model = AutoModelForMultipleChoice.from_pretrained(f'model_v{VER}')
trainer = Trainer(model=model)

In [24]:
df_test = df_valid.drop(columns=['answer'])

In [25]:
def test_preprocess(example):
    first_sentence = [ "[CLS] " + example['explanation'] ] * 4
    second_sentences = [" #### " + example['question'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCD']
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation=True,
                                  max_length=MAX_INPUT, add_special_tokens=False, padding='max_length')

    return tokenized_example

In [26]:
tokenized_test_dataset = Dataset.from_pandas(df_test).map(
        test_preprocess, remove_columns=['question', 'explanation', 'A', 'B', 'C', 'D'])
# tokenized_dataset_valid
test_predictions = trainer.predict(tokenized_test_dataset).predictions
predictions_as_ids = np.argsort(-test_predictions, 1)
predictions_as_answer_letters = np.array(list('ABCD'))[predictions_as_ids]
predictions_as_string = df_test['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [27]:
print(test_predictions[0])
print(predictions_as_ids[0])
print(predictions_as_answer_letters[0])
print(predictions_as_string[0])

[7.8164062 8.1171875 9.921875  5.4960938]
[2 1 0 3]
['C' 'B' 'A' 'D']
C B A


In [28]:
results = np.argmax(test_predictions, 1)
results_as_answer_letters = np.array(list('ABCD'))[results]
results_as_answer_letters

array(['C', 'C', 'B', 'D', 'A', 'B', 'A', 'A', 'B', 'C', 'B', 'C', 'B',
       'D', 'C', 'C', 'D', 'A', 'C', 'C', 'A', 'D', 'C', 'D', 'C', 'A',
       'A', 'B', 'D', 'D', 'C', 'A', 'C', 'B', 'C', 'B', 'C', 'C', 'B',
       'B', 'C', 'B', 'A', 'C', 'A', 'C', 'D', 'D', 'C', 'C', 'C', 'D',
       'A', 'B', 'C', 'A', 'B', 'D', 'D', 'A', 'D', 'A', 'C', 'D', 'C',
       'C', 'C', 'D', 'D', 'A', 'D', 'B', 'B', 'B', 'C', 'C', 'C', 'C',
       'A', 'A', 'A', 'A', 'A', 'C', 'A', 'C', 'D', 'A', 'C', 'B', 'A',
       'B', 'C', 'C', 'C', 'C', 'C', 'A', 'D', 'D', 'D', 'A', 'B', 'C',
       'B', 'C', 'D', 'D', 'C', 'A', 'B', 'A', 'A', 'C', 'C', 'C', 'D',
       'C', 'D', 'C'], dtype='<U1')

# Compute Validation Score


In [29]:
import numpy as np

def precision_at_k(r, k):
    """Precision at k"""
    assert k <= len(r)
    assert k != 0
    return sum(int(x) for x in r[:k]) / k

def MAP_at_3(predictions, true_items):
    """Score is mean average precision at 3"""
    U = len(predictions)
    map_at_3 = 0.0
    for u in range(U):
        user_preds = predictions[u].split()
        user_true = true_items[u]
        user_results = [1 if item == user_true else 0 for item in user_preds]
        for k in range(min(len(user_preds), 3)):
            map_at_3 += precision_at_k(user_results, k+1) * user_results[k]

    return map_at_3 / U

In [30]:
m = MAP_at_3(df_test.prediction.values, df_valid.answer.values)
print( 'CV MAP@3 =',m )

CV MAP@3 = 0.7847222222222221


In [32]:
ck_model = AutoModelForMultipleChoice.from_pretrained('/content/checkpoints_6/checkpoint-3300')

In [33]:
ck_model

DebertaV2ForMultipleChoice(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_aff

# Others

In [34]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [60]:
from google.colab import files

files.download('/content/checkpoints_6_1')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [58]:
!zip -r /content/checkpoints_6_1.zip /content/checkpoints_6_1
# !zip -r /content/model_v6_1.zip /content/model_v6_1

  adding: content/checkpoints_6_1/ (stored 0%)
  adding: content/checkpoints_6_1/checkpoint-1950/ (stored 0%)
  adding: content/checkpoints_6_1/checkpoint-1950/config.json (deflated 53%)
  adding: content/checkpoints_6_1/checkpoint-1950/optimizer.pt (deflated 8%)
  adding: content/checkpoints_6_1/checkpoint-1950/rng_state.pth (deflated 25%)
  adding: content/checkpoints_6_1/checkpoint-1950/scheduler.pt (deflated 55%)
  adding: content/checkpoints_6_1/checkpoint-1950/tokenizer.json (deflated 77%)
  adding: content/checkpoints_6_1/checkpoint-1950/training_args.bin (deflated 50%)
  adding: content/checkpoints_6_1/checkpoint-1950/model.safetensors


zip error: Interrupted (aborting)


In [57]:
!mv /content/checkpoints_6_1 /content/drive/MyDrive/RESEARCH/ZAIC_2023/Math
!mv /content/model_v6_1 /content/drive/MyDrive/RESEARCH/ZAIC_2023/Math

In [38]:
with open("math_test.json") as json_file:
    json_data = json.load(json_file)
    print(json_data['data'])

[{'id': '01-0203', 'question': 'Một cửa hàng đã bán 30% số hàng hiện có và thu được 15 000 000 đồng. Hỏi nếu bán hết hàng thì cửa hàng thu được bao nhiêu tiền?', 'choices': ['A. 4 500 000 đồng', 'B. 45 000 000 đồng', 'C. 50 000 000 đồng', 'D. 450 000 000 đồng']}, {'id': '01-0206', 'question': 'Một người đi xe đạp từ A lúc 7 giờ với vận tốc 12km/h. Đến 8 giờ một người đi xe máy cũng từ A đuổi theo người đi xe đạp với vận tốc 42km/h. Hỏi người đi xe máy đuổi kịp người đi xe đạp lúc mấy giờ?', 'choices': ['A. 24 phút', 'B. 1 giờ', 'C. 7 giờ 24 phút', 'D. 8 giờ 24 phút']}, {'id': '01-0207', 'question': 'Cạnh của hình lập phương gấp lên 2 lần thì diện tích xung quanh gấp lên mấy lần?', 'choices': ['A. 2 lần', 'B. 4 lần', 'C. 6 lần', 'D. 8 lần']}, {'id': '01-0209', 'question': 'Một thửa ruộng hình thang có đáy bé dài 8m, đáy lớn dài 12m. Kéo dài đáy lớn thêm 5m thì diện tích thửa ruộng tăng thêm 25m2. Hỏi diện tích thửa ruộng tăng thêm bao nhiêu phần trăm?', 'choices': ['A. 125m^{2}', 'B. 20

In [39]:
dict_one_hot_answer = {
    0: "A",
    1: "B",
    2: "C",
    3: "D"
}

list_id = []
list_question = []
list_A = []
list_B = []
list_C = []
list_D = []

for record in json_data['data']:
  id = record['id']
  question = record['question']
  choices = record['choices']
  try:
    explanation = record['explanation']
  except KeyError:
    explanation = "None"

  list_A.append(choices[0])
  list_B.append(choices[1])
  list_C.append(choices[2])
  try:
    list_D.append(choices[3])
  except IndexError:
    list_D.append("None")
  list_question.append(question)
  list_id.append(id)

In [40]:
data_df = pd.DataFrame(list(zip(list_id, list_question, list_A, list_B, list_C, list_D)),
                       columns=['id', 'question', 'A', 'B', 'C', 'D',])

data_df

,id,question,A,B,C,D
0,01-0203,Một cửa hàng đã bán 30% số hàng hiện có và thu...,A. 4 500 000 đồng,B. 45 000 000 đồng,C. 50 000 000 đồng,D. 450 000 000 đồng
1,01-0206,Một người đi xe đạp từ A lúc 7 giờ với vận tốc...,A. 24 phút,B. 1 giờ,C. 7 giờ 24 phút,D. 8 giờ 24 phút
2,01-0207,Cạnh của hình lập phương gấp lên 2 lần thì diệ...,A. 2 lần,B. 4 lần,C. 6 lần,D. 8 lần
3,01-0209,"Một thửa ruộng hình thang có đáy bé dài 8m, đá...",A. 125m^{2},B. 20%,C. 25%,D. 50%
4,01-0210,Một xe lửa vượt qua cái cầu dài 450m hết 45 gi...,A. 3m,B. 200m,C. 200m,D. 225m
...,...,...,...,...,...,...
184,01-0698,"Kết quả phép nhân 4,51 \times 10 là:",A. 451,"B. 4,51","C. 45,1",D. 45
185,01-0703,"Lớp 5/2 có 32 học sinh, trong đó có 12 học sin...",A. 375 %,"B. 37,5 %","C. 3,75 %","D. 0,375 %"
186,01-0715,Số thập phân thích hợp để điền vào chỗ chấm: 4...,"A. 45,62","B. 4,562","C. 456,2",D. 4562
187,01-0716,"Kết quả của X trong biểu thức: X \div 2,04 = ...","A. 3,03","B. 3,04","C. 3,05","D. 3,06"


In [41]:
def test_preprocess(example):
    second_sentences = [" #### " + example['question'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCD']
    tokenized_example = tokenizer(second_sentences, truncation=True,
                                  max_length=MAX_INPUT, add_special_tokens=False, padding='max_length')

    return tokenized_example

In [42]:
tokenized_test_dataset = Dataset.from_pandas(data_df).map(
        test_preprocess, remove_columns=['id', 'question', 'A', 'B', 'C', 'D'])
# tokenized_dataset_valid
test_predictions = trainer.predict(tokenized_test_dataset).predictions
predictions_as_ids = np.argsort(-test_predictions, 1)
predictions_as_answer_letters = np.array(list('ABCD'))[predictions_as_ids]
predictions_as_string = data_df['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]
predictions_as_string

Map:   0%|          | 0/189 [00:00<?, ? examples/s]

['B C D',
 'D C B',
 'A D B',
 'C D A',
 'D B C',
 'C A D',
 'A D C',
 'B C A',
 'C A B',
 'A C D',
 'C B D',
 'B A D',
 'C D B',
 'C B D',
 'A D B',
 'A B D',
 'A D B',
 'A B D',
 'A D C',
 'A C B',
 'C B A',
 'D C A',
 'B A D',
 'D C B',
 'A C B',
 'B D C',
 'C D B',
 'B D A',
 'C D B',
 'A B D',
 'D B C',
 'D A B',
 'D B C',
 'B D A',
 'B D A',
 'A B D',
 'C D A',
 'B A C',
 'B D C',
 'A B C',
 'A C B',
 'A B D',
 'D A C',
 'A B C',
 'A B C',
 'C B A',
 'B D C',
 'C B A',
 'A B D',
 'B D C',
 'A C B',
 'C A B',
 'A B C',
 'C B D',
 'C B D',
 'A B C',
 'B C D',
 'A B C',
 'A B C',
 'A B D',
 'A B D',
 'A C B',
 'A C B',
 'A D B',
 'B A C',
 'C B D',
 'A B C',
 'D A C',
 'C B D',
 'B D A',
 'A C B',
 'B C A',
 'A D C',
 'C D B',
 'B A D',
 'B C A',
 'C A D',
 'A C B',
 'A D B',
 'D C B',
 'C A B',
 'C B A',
 'C B A',
 'C B A',
 'C B A',
 'C B A',
 'C D A',
 'C D B',
 'D A C',
 'A B C',
 'A B C',
 'A D B',
 'C B D',
 'D A B',
 'D C B',
 'B A D',
 'D C A',
 'A D C',
 'A D C',
 'C D A',


In [47]:
results = np.argmax(test_predictions, 1)
results_as_answer_letters = np.array(list('ABCD'))[results]
results_to_ids = results_as_answer_letters.tolist()

In [45]:
final_results = [data_df[ans][idx] for idx, ans in enumerate(results_to_ids)]

In [50]:
data_df['answer'] = final_results
data_df

,id,question,A,B,C,D,prediction,answer
0,01-0203,Một cửa hàng đã bán 30% số hàng hiện có và thu...,A. 4 500 000 đồng,B. 45 000 000 đồng,C. 50 000 000 đồng,D. 450 000 000 đồng,B C D,A. 4 500 000 đồng
1,01-0206,Một người đi xe đạp từ A lúc 7 giờ với vận tốc...,A. 24 phút,B. 1 giờ,C. 7 giờ 24 phút,D. 8 giờ 24 phút,D C B,A. 24 phút
2,01-0207,Cạnh của hình lập phương gấp lên 2 lần thì diệ...,A. 2 lần,B. 4 lần,C. 6 lần,D. 8 lần,A D B,C. 6 lần
3,01-0209,"Một thửa ruộng hình thang có đáy bé dài 8m, đá...",A. 125m^{2},B. 20%,C. 25%,D. 50%,C D A,B. 20%
4,01-0210,Một xe lửa vượt qua cái cầu dài 450m hết 45 gi...,A. 3m,B. 200m,C. 200m,D. 225m,D B C,A. 3m
...,...,...,...,...,...,...,...,...
184,01-0698,"Kết quả phép nhân 4,51 \times 10 là:",A. 451,"B. 4,51","C. 45,1",D. 45,B C A,D. 45
185,01-0703,"Lớp 5/2 có 32 học sinh, trong đó có 12 học sin...",A. 375 %,"B. 37,5 %","C. 3,75 %","D. 0,375 %",B A D,"C. 3,75 %"
186,01-0715,Số thập phân thích hợp để điền vào chỗ chấm: 4...,"A. 45,62","B. 4,562","C. 456,2",D. 4562,B D C,"A. 45,62"
187,01-0716,"Kết quả của X trong biểu thức: X \div 2,04 = ...","A. 3,03","B. 3,04","C. 3,05","D. 3,06",A C D,"B. 3,04"


In [51]:
answer = data_df[['id', 'answer']]
answer.to_csv('/content/result.csv')

In [52]:
!mv /content/result.csv /content/drive/MyDrive/RESEARCH/ZAIC_2023/Math

In [55]:
!mv /content/checkpoints_6_1.zip /content/drive/MyDrive/RESEARCH/ZAIC_2023/Math